1.1

In [1]:
from pyspark import SparkConf, SparkContext
from pyspark.sql import SQLContext
import pyspark.sql.functions as F
import urllib 
import json

In [2]:
sc = SparkContext.getOrCreate()
sqlContext = SQLContext(sc)

In [3]:
path = "shakespeare_full.json"
#load json file from path
df2 = sqlContext.read.json(path)

#show the first 20 record of df2
df2.show()

+---+-------+-----------+---------+-------------+-------------+--------------------+-----+
|_id|line_id|line_number|play_name|      speaker|speech_number|          text_entry| type|
+---+-------+-----------+---------+-------------+-------------+--------------------+-----+
|  0|      1|           | Henry IV|             |         null|               ACT I|  act|
|  1|      2|           | Henry IV|             |         null|SCENE I. London. ...|scene|
|  2|      3|           | Henry IV|             |         null|Enter KING HENRY,...| line|
|  3|      4|      1.1.1| Henry IV|KING HENRY IV|            1|So shaken as we a...| line|
|  4|      5|      1.1.2| Henry IV|KING HENRY IV|            1|Find we a time fo...| line|
|  5|      6|      1.1.3| Henry IV|KING HENRY IV|            1|And breathe short...| line|
|  6|      7|      1.1.4| Henry IV|KING HENRY IV|            1|To be commenced i...| line|
|  7|      8|      1.1.5| Henry IV|KING HENRY IV|            1|No more the thirs...| line|

In [4]:
#group data by 'speaker' and show the seaker and count of records
df2.groupby('speaker').agg(F.count("speaker").alias("count")).show()

+-----------------+-----+
|          speaker|count|
+-----------------+-----+
|       EUPHRONIUS|   16|
|Third Conspirator|   12|
|            PETER|   63|
|  First Gentleman|  284|
|           AEGEON|  150|
|        DONALBAIN|   10|
|        LYCHORIDA|   11|
|          QUINTUS|   30|
|           AENEAS|  153|
|           Porter|   97|
|          RUTLAND|   26|
|              NYM|   78|
|   LORD FITZWATER|   27|
|         CARDINAL|  120|
|       Attendants|    2|
|       ANTIPHOLUS|    6|
|    Third Servant|   31|
|        ANNE PAGE|   31|
|        Moonshine|    6|
|       SIR ANDREW|  155|
+-----------------+-----+
only showing top 20 rows



In [5]:
#lowe case the 'text_entry' and search for those whiche contains 'sometimes'
# and their 'line_number' start with '1.1.'
# then, it showes the first 20 columns of "_id", "speaker", "line_number", "text_entry"
df_filter = df2.filter(F.lower(df2.text_entry).contains('sometimes') & df2.line_number.startswith('1.1.') )\
.select("_id", "speaker", "line_number", "text_entry")
df_filter.show(truncate=False)


+-----+---------+-----------+----------------------------------------------------+
|_id  |speaker  |line_number|text_entry                                          |
+-----+---------+-----------+----------------------------------------------------+
|18634|PHILO    |1.1.63     |Sir, sometimes, when he is not Antony,              |
|32496|HORATIO  |1.1.59     |Did sometimes march? by heaven I charge thee, speak!|
|61534|BASSANIO |1.1.166    |Of wondrous virtues: sometimes from her eyes        |
|64418|SLENDER  |1.1.240    |A justice of peace sometimes may be beholding to his|
|75845|ANTIOCHUS|1.1.34     |Yon sometimes famous princes, like thyself,         |
+-----+---------+-----------+----------------------------------------------------+



In [8]:
# filter the data where 'speaker' is 'DONALBAIN'
# and select the lenght of character in 'text_entry'
result_len_text = df2.filter(df2.speaker.contains('DONALBAIN')).select(F.length('text_entry').alias('length'))
lst_len_text = [row['length'] for row in result_len_text.collect()]
#show a list of intergers:
print(lst_len_text)

[14, 47, 15, 45, 11, 28, 36, 43, 49, 18]


In [9]:
# filter the data where 'speaker' is 'DONALBAIN'
# and select the '_id' and the number of words in 'text_entry'
# for calculate the num of words, at first,punctuation symbols are removed by regular expresion 'regexp_replace'
# and then split the 'text_entry' to a list of words and the count of words is calculated by using 'size' method
result_word_count = df2.filter(df2.speaker.contains('DONALBAIN'))\
.select('_id',F.size(F.split(F.regexp_replace(df2.text_entry,'[^\w\s]' ,''),' ')).alias('count'))

#Generate a list of pairs (key, value)
lst_word_count = [(row[0],row[1] ) for row in result_word_count.collect()]
print(lst_word_count)

[(56668, 3), (56698, 9), (56699, 3), (56700, 9), (56701, 3), (56702, 6), (56723, 6), (56724, 9), (56725, 9), (56726, 3)]
